<a href="https://colab.research.google.com/github/IkerZha0401/QM2_Final_PJ/blob/main/map_protest_intensity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install Basemap
!pip install ipyleaflet
import requests
import pandas as pd
import os
from google.colab import files
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pylab
from datetime import datetime

%matplotlib inline
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

def download_and_process_data(url):
  filename = os.path.basename(url)
  response = requests.get(url)

  with open(filename, 'wb') as file:
      file.write(response.content)

  df = pd.read_csv(filename)

  return df


df = download_and_process_data("https://raw.githubusercontent.com/IkerZha0401/QM2_Final_PJ/refs/heads/main/ProtestV1.csv")
print(df.head())



In [2]:
df['event_date']=pd.to_datetime(df['event_date'])
df['event_date'] = df['event_date'].dt.strftime('%Y-%m-%d')
df['event_date']=pd.to_datetime(df['event_date'])
df['event_date'] = df['event_date'].dt.strftime('%Y-%m-%d')
print(df.head())

  event_id_cnty  event_date  year  country  latitude  longitude
0       IRN6075  2020-01-31  2020     Iran   32.6525    51.6746
1       ALG8120  2020-01-31  2020  Algeria   36.7500     5.0833
2       ALG8123  2020-01-31  2020  Algeria   36.3800     3.9014
3       ALG8124  2020-01-31  2020  Algeria   36.7664     3.4772
4       ALG8126  2020-01-31  2020  Algeria   36.3650     6.6147


In [3]:
import pandas as pd
import plotly.express as px

df['event_date'] = pd.to_datetime(df['event_date'])
df = df[(df['event_date'] >= "2010-12-17") & (df['event_date'] <= "2020-01-31")]


df['year_month'] = df['event_date'].dt.to_period('M').astype(str)
df_monthly = (
    df.groupby(['year_month', 'country', 'latitude', 'longitude'], as_index=False)
      .size()
      .rename(columns={'size': 'protest_count'})
)


ordered_months = [
    dt.strftime('%Y-%m')
    for dt in pd.date_range(start='2010-12-01', end='2020-01-31', freq='MS')
]


df_monthly = df_monthly[df_monthly['year_month'].isin(ordered_months)]


fig = px.scatter_mapbox(
    df_monthly,
    lat='latitude',
    lon='longitude',
    hover_name='country',
    hover_data=['protest_count'],
    size='protest_count',
    color='protest_count',
    color_continuous_scale='Reds',
    range_color=(50, df_monthly['protest_count'].max()),
    zoom=3,
    center={'lat': 27, 'lon': 0},
    mapbox_style='open-street-map',
    animation_frame='year_month',
    category_orders={'year_month': ordered_months},
    size_max=30,
    labels={
        'protest_count': 'Numbers of Protest'
    }
)

fig.update_layout(
    transition={'duration': 0},
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig.show()